In [1]:
%pip install clemcore==3.3.5 pettingzoo>=1.25.0 langchain>=0.1.0 langchain-openai>=0.0.5 langchain-core>=0.1. ipywidgets>=8.1.0 jupyter_client>=8.7.0 httpx>=0.24.0 

Note: you may need to restart the kernel to use updated packages.


The process cannot access the file because it is being used by another process.


In [ ]:
import os

# Specify the game name here (this code can be adapted to any 2-player game)
GAME_NAME = "taboo"

# Local clone location of the clembench repository

#CLEMBENCH_HOME =r"C:\Users\white\clembench-im" 
CLEMBENCH_HOME =r"C:\Users\white\Desktop\agents_experiments\clemcore-feat-playpen-examples\clemcore"
print(CLEMBENCH_HOME)

# Expose CLEMBENCH_HOME so the clem framework can find the games
os.environ["CLEMBENCH_HOME"] = CLEMBENCH_HOME

In [2]:
# Clone the clembench repo (safe to re-run; git will warn if it already exists)
#!git clone https://github.com/clp-research/clembench $CLEMBENCH_HOME

# Install the requirements into the Python kernel
%pip install -r $CLEMBENCH_HOME/requirements.txt

# Make tqdm usable in Jupyter notebooks
%pip install --upgrade ipywidgets jupyter_client

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'C:\\Users\\white\\Desktop\\agents_experiments\\clemcore-feat-playpen-examples\\clemcore/requirements.txt'


Note: you may need to restart the kernel to use updated packages.


In [3]:
# für den Fall der Fälle
#!pip install --upgrade clemcore==3.3.5

In [4]:
from clemcore.backends import ModelRegistry

registry = ModelRegistry.register("clp-chat", backend="openai_compatible", model_id="Qwen/Qwen3-Coder-30B-A3B-Instruct-FP8")
registry.get_first_model_spec_that_unify_with("clp-chat")

2026-01-05 13:24:18,967 - clemcore.backends.model_registry - WARNING - Package lookup failed with exception: 'charmap' codec can't decode byte 0x81 in position 128272: character maps to <undefined>


ModelSpec({'model_name': 'clp-chat', 'backend': 'openai_compatible', 'lookup_source': 'C:\\Users\\white\\Desktop\\agents_experiments\\playpen\\model_registry.json', 'model_id': 'Qwen/Qwen3-Coder-30B-A3B-Instruct-FP8'})

In [ ]:
API_KEY = ""
ORGANIZATION = "<insert-organisation-here>"
BASE_URL = ""

In [6]:
from clemcore.backends import KeyRegistry
KeyRegistry.register("openai_compatible", api_key=API_KEY,organisation=ORGANIZATION,base_url=BASE_URL, force_cwd=True)

KeyRegistry(file='C:\Users\white\Desktop\agents_experiments\playpen\key.json', backends=[alephalpha, anthropic, cohere, huggingface, openai, openai_compatible])

In [13]:
from langchain_core.messages import AIMessage
from playpen.agents import ClemAgent, ClemObservation
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
import re


def get_guess_from_content(content: str) -> str:
    """Extract the last line that starts with CLUE: from the content string."""
    print(content)
    matches = re.findall(r'(CLUE:\s*.+)', content)
    last_guess = matches[-1] if matches else "No clue"
    return last_guess
        
class MyAgenticDescriber(ClemAgent):

    def __init__(self):
        super().__init__()
        self.model = ChatOpenAI(
            model="openai/gpt-3.5-turbo",
            base_url="https://openrouter.ai/api/v1",
            api_key="",  #deleted for security reasons
            temperature=0,
        )
        
    def act(self, last: ClemObservation) -> str:

        agent = create_agent(
              self.model,
              tools=[],
              system_prompt="You are a helpful assistant",
          )

        result = agent.invoke(
    {"messages": [{"role": "user", "content": last.content}]})

        final_ai_message = next(msg for msg in reversed(result["messages"]) if isinstance(msg, AIMessage))


        response_text = final_ai_message.content
        return response_text


describer = MyAgenticDescriber()

In [14]:
from clemcore.clemgame import gym_env
from clemcore.clemgame.envs.pettingzoo import env, gym_env
from clemcore.backends import load_model



game_env = gym_env(
    GAME_NAME,
    learner_agent="Player_1",
    env_agents={"Player_2": load_model("clp-chat")}
)

2026-01-05 13:25:48,938 - clemcore.backends - WARNING - No generation arguments provided, using default values: {'temperature': 0.0, 'max_tokens': 300}.
2026-01-05 13:25:48,943 - clemcore.backends.model_registry - WARNING - Package lookup failed with exception: 'charmap' codec can't decode byte 0x81 in position 128272: character maps to <undefined>
2026-01-05 13:25:48,947 - clemcore.backends - INFO - Found registered model spec that unifies with {"model_name":"clp-chat"} -> {'model_name': 'clp-chat', 'backend': 'openai_compatible', 'lookup_source': 'C:\\Users\\white\\Desktop\\agents_experiments\\playpen\\model_registry.json', 'model_id': 'Qwen/Qwen3-Coder-30B-A3B-Instruct-FP8'}
2026-01-05 13:25:48,950 - clemcore.backends - INFO - Found registry entry for backend openai_compatible -> {'backend': 'openai_compatible', 'file_name': 'openai_compatible_api.py', 'file_path': 'C:\\Users\\white\\Desktop\\agents_experiments\\clemcore-feat-playpen-examples\\clemcore\\backends\\openai_compatible_a

In [15]:
obs, info = game_env.reset()
termination = False
context_response_pairs: list[tuple] = []
while not termination:
    action = describer(obs)
    context_response_pairs.append((obs, action))
    obs, reward, termination, truncation, info = game_env.step(action)
    print(obs, action)

# Reset agent state between episodes (if it maintains any internal memory)
describer.reset()

print(f"Episode took these {len(context_response_pairs)} steps:")
print("-" * 20)
for idx, (context, response) in enumerate(context_response_pairs):
    print(f"Step {idx}:")
    print(f"Describer <- Context:", context)
    print(f"Describer -> Response:", response)
    print("-" * 20)


{'role': 'user', 'content': 'CLUE: (3) My third clue is ...'} Is there something specific you would like me to guess about trees?
Episode took these 1 steps:
--------------------
Step 0:
Describer <- Context: {'role': 'user', 'content': 'GUESS: tree'}
Describer -> Response: Is there something specific you would like me to guess about trees?
--------------------
